https://docs.julialang.org/en/release-0.4/manual/unicode-input/

## DTW


In [1]:
δ(a,b) = (a - b)^2
# first(x) = x[1] firstは元からあるのでいらない
second(x) = x[2]

function minVal(v₁, v₂, v₃)
#    if first(v₁) ≦ minimum([first(v₂), first(v₃)])
    if first(v₁) <= minimum([first(v₂), first(v₃)])
        return v₁, 1
    elseif first(v₂) <= first(v₃)
        return v₂, 2
    else
        return v₃, 3
    end
end

function DTW(A, B)
    S = length(A)
    T = length(B)
    m = Matrix(S, T)
    m[1, 1] = [δ(A[1], B[1]), (0,0)]
    for i in 2:S
        m[i,1] = [m[i-1, 1][1] + δ(A[i], B[1]), [i-1, 1]]
    end
    for j in 2:T
        m[1,j] = [m[1, j-1][1] + δ(A[1], B[j]), [1, j-1]]
    end
    for i in 2:S
        for j in 2:T
            min, index = minVal(m[i-1,j], m[i,j-1], m[i-1,j-1])
            indexes = [[i-1, j], [i, j-1], [i-1, j-1]]
            m[i,j] = first(min) + δ(A[i],B[j]), indexes[index]
        end
    end
    return m
end

DTW (generic function with 1 method)

In [2]:
T = 150
A = sin((1:T)/10)
B = sin((1:T)/10 + pi*0.4)
C = [0 for i in 1:T]

150-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [3]:
m = DTW(A, B)[150,150]

(9.3585894670215,[150,149])

In [4]:
DTW(A, C)[150,150][1]

77.67327710659531

## Stochastic Block Model

In [2]:
# Pkg.add("Einsum")
using Einsum

In [7]:
K = 8
α = 6
a₀ = b₀ = 0.5

α₁ = α₂ = transpose(ones(K) * α)
logΓ = lgamma

lgamma (generic function with 8 methods)

In [27]:
# Pkg.add("StatsBase")  # Only do this once, obviously
using StatsBase

INFO: No packages to install, update or remove
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of StatsBase
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [81]:
function onehot(i, K)
    ret = zeros(K)
    ret[i] = 1
    return ret
end

function m(x)
    return sum(x,1)
end

m (generic function with 1 method)

In [4]:
𝕀 # \BbbI 

𝕀 (generic function with 1 method)

In [79]:
function update_z₁(X, 𝕀z₁, 𝕀z₂)
    N₁, N₂ = size(X)
    m₁ = m(𝕀z₁)
    m₂ = m(𝕀z₂)
    
    for i in 1:N₁
        @einsum n⁺[k,l] := X[i,j] * 𝕀z₁[i,k] * 𝕀z₂[j,l]
        @einsum n⁻[k,l] := (ones(X)[i,j] - X[i,j]) * 𝕀z₁[i,k] * 𝕀z₂[j,l]
    
        m̂₁ = m(𝕀z₁) - transpose(𝕀z₁[i,:])
        @einsum Σ⁺x𝕀z₂[i,l] := X[i,j] * 𝕀z₂[j,l]
        @einsum Σ⁻x𝕀z₂[i,l] := (ones(X)[i,j] - X[i,j]) * 𝕀z₂[j,l]
        @einsum n̂⁺[k,l] := n⁺[k,l] - 𝕀z₁[i,k] * Σ⁺x𝕀z₂[i,l]
        @einsum n̂⁻[k,l] := n⁻[k,l] - 𝕀z₁[i,k] * Σ⁻x𝕀z₂[i,l]
        
        α̂₁ = α₁ + m̂₁
        â = a₀ + n̂⁺
        b̂ = b₀ + n̂⁻

        temp⁺ = zeros(â)
        temp⁻ = zeros(â)
        temp = zeros(â)
        for j in 1:size(temp⁺)[1]
            temp⁺[j,:] = Σ⁺x𝕀z₂[i,:]
            temp⁻[j,:] = Σ⁻x𝕀z₂[i,:]
            temp[j,:] = sum(𝕀z₂,1)
        end
        @einsum p_z₁[k,l] := exp(logΓ(â + b̂)-logΓ(â)-logΓ(b̂) 
            + logΓ(â+temp⁺)+logΓ(b̂+temp⁻)-logΓ(â+b̂+temp))[k,l]
        p_z₁ = α̂₁ .* transpose(prod(p_z₁, 2))
        p_z₁ /= sum(p_z₁)
        
        𝕀z₁[i,:] = onehot(sample(1:K, Weights(p_z₁[:])), K)
    end
    return 𝕀z₁
end

update_z₁ (generic function with 1 method)

In [88]:
function update_z₂(X, 𝕀z₁, 𝕀z₂)
    N₁, N₂ = size(X)
    m₁ = m(𝕀z₁)
    m₂ = m(𝕀z₂)
    
    for i in 1:N₂
        println(𝕀z₂)

        @einsum n⁺[k,l] := X[i,j] * 𝕀z₁[i,k] * 𝕀z₂[j,l]
        @einsum n⁻[k,l] := (ones(X)[i,j] - X[i,j]) * 𝕀z₁[i,k] * 𝕀z₂[j,l]
    
        m̂₂ = m(𝕀z₂) - transpose(𝕀z₂[i,:])
        @einsum Σ⁺x𝕀z₁[i,l] := X[i,j] * 𝕀z₁[j,l]
        @einsum Σ⁻x𝕀z₁[i,l] := (ones(X)[i,j] - X[i,j]) * 𝕀z₁[j,l]
        @einsum n̂⁺[k,l] := n⁺[k,l] - 𝕀z₂[i,k] * Σ⁺x𝕀z₁[i,l]
        @einsum n̂⁻[k,l] := n⁻[k,l] - 𝕀z₂[i,k] * Σ⁻x𝕀z₁[i,l]
        
        α̂₂ = α₂ + m̂₂
        â = a₀ + n̂⁺
        b̂ = b₀ + n̂⁻

        temp⁺ = zeros(â)
        temp⁻ = zeros(â)
        temp = zeros(â)
        for j in 1:size(temp⁺)[1]
            temp⁺[j,:] = Σ⁺x𝕀z₁[i,:]
            temp⁻[j,:] = Σ⁻x𝕀z₁[i,:]
            temp[j,:] = sum(𝕀z₁,1)
        end
        @einsum p_z₂[k,l] := exp(logΓ(â + b̂)-logΓ(â)-logΓ(b̂) 
            + logΓ(â+temp⁺)+logΓ(b̂+temp⁻)-logΓ(â+b̂+temp))[k,l]
        p_z₂ = α̂₂ .* transpose(prod(p_z₂, 2))
        p_z₂ /= sum(p_z₂)
        
        𝕀z₂[i,:] = onehot(sample(1:K, Weights(p_z₂[:])), K)
    end
    return 𝕀z₂
end

update_z₂ (generic function with 1 method)

In [9]:
using DataFrames
using CSV

In [10]:
data = readtable("./bi_data.csv")
X = hcat(data.columns...)

120×120 DataArrays.DataArray{Int64,2}:
 0  0  1  0  0  0  0  1  0  1  1  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  1  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  1  0  1  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  1  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  1  0  0  0  0  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  1  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  1  1  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  

In [11]:
𝕀z₁ = zeros(size(X)[1],K)
𝕀z₁[:,1] = 1
𝕀z₂ = zeros(size(X)[1],K)
𝕀z₂[:,1] = 1

1

In [89]:
a = update_z₂(X, 𝕀z₁, 𝕀z₂)

[1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0

LoadError: InterruptException:

In [80]:
a = update_z₁(X, 𝕀z₁, 𝕀z₂)

1[0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0]
2[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
3[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
4[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
5[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
6[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
7[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
8[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
9[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
10[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
11[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
12[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
13[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
14[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
15[0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0]
16[0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0]
17[0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0]
18[0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0]
19[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
20[0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0]
21[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
22[0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0]
23[0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0]
24[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
25[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
26[0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0]
27[0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0]
28[1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
2

LoadError: InterruptException:

関数直した時に，その関数を読んでる関数ももう一度実行しないといけない．